In [16]:
import numpy as np
import numpy.matlib as mlt
import mne
import statistics 
from pathlib import Path
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
import pandas as pd
from mne.filter import filter_data 
%matplotlib inline

Code snippet for file processing, example of using below.

In [17]:
def microvolts_to_volts(value):
    """
    Since openBCI writes data into micro volts and mne works with volts we
    will need to convert the data later.
    :param value: single micro volts value
    :return: same value in volts
    """
    return float(value) / 1000


def load_openbci_file(filename, ch_names=None, skiprows=0, max_rows=0):
    """
    Load data from OpenBCI file into mne RawArray for later use
    :param filename: filename for reading in form of relative path from working directory
    :param ch_names: dictionary having all or some channels like this:
            {"fp1":1, "fp2":2, "c3":3, "c4":4, "o1":5, "o2":6, "p3":7, "p4":8}
            Key specifies position on head using 10-20 standard and
            Value referring to channel number on Cyton BCI board
    :return: RawArray class of mne.io library
    """
    if ch_names is None:
        ch_names = {"fp1":1, "fp2":2, "c3":3, "c4":4, "o1":5, "o2":6, "p3":7, "p4":8}

    # Converter of BCI file to valuable data
    converter = {i: (microvolts_to_volts if i < 12 else lambda x: str(x).split(".")[1][:-1])
                 for i in range(0, 13)}

    info = mne.create_info(
        ch_names=list(ch_names.keys()),
        ch_types=['eeg' for i in range(0, len(ch_names))],
        sfreq=250,
        montage='standard_1020'
    )
    data = np.loadtxt(filename, comments="%", delimiter=",",
                      converters=converter, skiprows=skiprows, max_rows=max_rows).T
    data = data[list(ch_names.values())]
    data = filter_data(data, 250, l_freq=2, h_freq=50)
#     print ('data type', type(data), '  shape  ' , data.shape)
    return mne.io.RawArray(data, info)


def create_epochs(raw_data, duration=1):
    """
    Chops the RawArray onto Epochs given the time duration of every epoch
    :param raw_data: mne.io.RawArray instance
    :param duration: seconds for copping
    :return: mne Epochs class
    """
    events = mne.make_fixed_length_events(raw_data, duration=duration)
    epochs = mne.Epochs(raw_data, events, preload=True)
    return epochs

def create_epochs_with_baseline(raw_data, baseline, duration=1):
    """
    Chops the RawArray onto Epochs given the time duration of every epoch
    :param raw_data: mne.io.RawArray instance
    :param duration: seconds for copping
    :return: mne Epochs class
    """
    events = mne.make_fixed_length_events(raw_data, duration=duration)
    epochs = mne.Epochs(raw_data, events, preload=True, baseline=baseline)
    return epochs


def get_files(dir='.', pattern='*.txt'):
    """
    Loading files from given directory with specified pattern.
    :param dir: Lookup directory
    :param pattern: Pattern for files. Default *.txt for loading raw BCI files
    :return: array of file paths
    """
    # Specifying files directory, select all the files from there which is txt
    datadir = Path(dir).glob(pattern)
    # Transferring generator into array of file paths
    return [x for x in datadir]

In [129]:
def get_sample_data(path, regx, skiprow=10000, max_row=133000):
    files = get_files(path, regx)
    ch_names = {"fp2":1, "fp1":2, "f4":3, "f3":4, "c4":5, "c3":6, "o2":7, "o1":8}
    raw_data = []
    raw_baselines = []
#     file = files[-1]
#     print(file)
#     raw_data.append(load_openbci_file(file, ch_names=ch_names, skiprows=skiprow, max_rows=max_row))
    
#     return create_epochs(raw_data[0])  
    
    for file in files:
        raw_data.append(load_openbci_file(file, ch_names=ch_names, skiprows=skiprow, max_rows=max_row))
        baseline_subj = str(file).split('_')[2]
        raw_baselines.append('*' + baseline_subj + '*Baseline*')
        
    real_data_series = [create_epochs(raw) for raw in raw_data]
    return real_data_series, raw_baselines

Defining Constants 

In [19]:
 # TAKE ONE RANDOM RECORDING FOR PLOTTING
n_channels = 8
SAMPLE_FREQ = 250

Next applying ICA and print the filters.

Note: Filter ID may not be the same as channel ID.


In [20]:
def transform_ICA(sample_data):
    ica = ICA()
    ica.fit(sample_data)
    return ica.apply(sample_data) # Transform recording into ICA space

# Convert data into vector
Try different approaches. Next is an attempt to see overall recording waves.
Free to change everything next.

In [21]:
def remove_epochs(data_epochs):
    dat = data_epochs.get_data()
    data = np.zeros( (dat.shape[0] * dat.shape[2], 8) )
    n_epoch = len(dat)
    n_in_epoch = dat.shape[2]
    for i in range(n_epoch):
        data[i*n_in_epoch:i*n_in_epoch + n_in_epoch] = dat[i].T
    
    return data

In [22]:
def build_table(data):
    # Get real amplitudes of FFT (only in postive frequencies)
    fft_vals = np.absolute(np.fft.rfft(data))

    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(data), 1.0/SAMPLE_FREQ)
    # Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    # Take the mean of the fft amplitude for each EEG band
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(fft_vals[freq_ix])

    # Plot the data (using pandas here cause it's easy)

    df = pd.DataFrame(columns=['band', 'val'])
    df['band'] = eeg_bands.keys()
    df['val'] = [eeg_band_fft[band] for band in eeg_bands]
    ax = df.plot.bar(x='band', y='val', legend=False)
    ax.set_xlabel("EEG band")
    ax.set_ylabel("Mean band Amplitude")
    return df

In [23]:
sample_data = get_sample_data('Dataset/', '*Session1_Coding.txt')
sample_data = transform_ICA(sample_data)
#ica returns mne.epochs.Epochs
# table_of_coding = build_table(remove_epochs(sample_data))

Dataset\OpenBCI-RAW-2019-06-26_20-41-48_Subject7_Session1_Coding.txt
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 413 samples (1.652 sec)

Creating RawArray with float64 data, n_channels=8, n_times=133000
    Range : 0 ... 132999 =      0.000 ...   531.996 secs
Ready.
532 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 532 events and 176 original time points ...
1 bad epochs dropped
Fitting ICA to data using 8 channels (please be patient, this may take a while)
I

In [24]:
# table_of_coding.iloc[2, 1] / table_of_coding.iloc[4, 1]

In [25]:
# table_of_coding

In [41]:
def vectorize(sample_data, waves1='Alpha', waves2='Beta'):
    vector = []
    # Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}
    
    
    ch_names = {"fp2":0, "fp1":1, "f4":2, "f3":3, "c4":4, "c3":5, "o2":6, "o1":7}
    
    
    analysis_data = np.zeros( (sample_data.shape[0], sample_data.shape[1]//2, sample_data.shape[2]) )
    i = 0
    
    # Calculate hemisphere difference ratio left / right
    for sample_epoch in sample_data:
        analysis_data[i][0] = (sample_epoch[1] - sample_epoch[0]) / (sample_epoch[1] + sample_epoch[0])
        analysis_data[i][1] = (sample_epoch[3] - sample_epoch[2]) / (sample_epoch[3] + sample_epoch[2])
        analysis_data[i][2] = (sample_epoch[5] - sample_epoch[4]) / (sample_epoch[5] + sample_epoch[4])
        analysis_data[i][3] = (sample_epoch[7] - sample_epoch[6]) / (sample_epoch[7] + sample_epoch[6])
        i+=1
    
#     vector = np.zeros((i,1))
    for epoch in analysis_data:
    
     # Get real amplitudes of FFT (only in postive frequencies)
        fft_vals = np.absolute(np.fft.rfft(epoch.T))

        # Get frequencies for amplitudes in Hz
        fft_freq = np.fft.rfftfreq(len(epoch.T), 1.0/SAMPLE_FREQ)
        eeg_band_fft = dict()
    
        for band in eeg_bands:
            freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                               (fft_freq <= eeg_bands[band][1]))[0]
            eeg_band_fft[band] = np.mean(fft_vals[freq_ix])
    
        vector.append(eeg_band_fft[waves1] / eeg_band_fft[waves2])

    return np.array(vector)

In [54]:
def apply_absolute_baseline(data, baseline):
    baseline = vectorize(baseline)
    baseline = mlt.repmat(baseline, 1, int(np.ceil(len(data)/len(baseline))))[0]
    base = baseline[:len(data)]
#     print('Base shape', base.shape)
#     print('Data shape', data.shape)
    data = data.reshape(9, int(np.ceil(len(data)/9)))
    base = base.reshape(9, int(np.ceil(len(base)/9)))
#     print('Base shape', base.shape)
#     print('Data shape', data.shape)
    data = np.mean(data.T, axis=1)
    base = np.mean(base.T, axis=1)
#     print('Base shape', base.shape)
#     print('Data shape', data.shape)
    
    return data - base

In [117]:
def compare_baseline(subj_data, subj_name):
    base = get_files('Dataset/', subj_name)
    ch_names = {"fp2":1, "fp1":2, "f4":3, "f3":4, "c4":5, "c3":6, "o2":7, "o1":8}
    base = load_openbci_file(base[0], ch_names=ch_names, skiprows=1000, max_rows=20000)
    base = create_epochs(base).get_data()
    
    subj_data = vectorize(subj_data.get_data())
    base = vectorize(base)
    base = mlt.repmat(base, 1, int(np.ceil(len(subj_data)/len(base))))[0]
    base = base[:len(subj_data)]
    
    return spatial.distance.euclidean(subj_data, base)

In [94]:
def do_baseline(subj_data, subj_name):
    base = get_files('Dataset/', subj_name)
    ch_names = {"fp2":1, "fp1":2, "f4":3, "f3":4, "c4":5, "c3":6, "o2":7, "o1":8}
    base = load_openbci_file(base[0], ch_names=ch_names, skiprows=1000, max_rows=20000)
    base = create_epochs(base).get_data()
    
    subj_data = vectorize(subj_data.get_data())
    
#     print(base.shape)
#     print(subj7_data.shape)

    return apply_absolute_baseline(subj_data, base)
    

In [56]:
from sklearn.preprocessing import normalize
# coding_vec = normalize(vectorize(sample_data.get_data(), 'Alpha', 'Beta')[:,np.newaxis], axis=0).ravel()

In [96]:
from scipy import spatial 

corrected_coding = do_baseline(sample_data, '*Subject7*Session1*Baseline*') 

temp_code_data = get_sample_data('Dataset/', '*Subject7*Session2*Coding*')
temp_code_data = transform_ICA(temp_code_data)

corrected_coding2 = do_baseline(temp_code_data, '*Subject7*Session2*Baseline*')
# meditation_vec = normalize(vectorize(sample_data, 'Alpha', 'Beta')[:,np.newaxis], axis=0).ravel()

# spatial.distance.euclidean(coding_vec, meditation_vec)

Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 413 samples (1.652 sec)

Creating RawArray with float64 data, n_channels=8, n_times=20000
    Range : 0 ... 19999 =      0.000 ...    79.996 secs
Ready.
80 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 80 events and 176 original time points ...
1 bad epochs dropped
Base shape (531,)
Data shape (531,)
Base shape (9, 59)
Data shape (9, 59)
Base shape (59,)
Data shape (59,)
Dataset\OpenBCI-RAW-2019-06-06_21-41-23_Subjec

In [97]:
spatial.distance.correlation(normalize(corrected_coding[:,np.newaxis], axis=0).ravel(), normalize(corrected_coding2[:,np.newaxis], axis=0).ravel())

1.2173146277534739

In [98]:
code_data = get_sample_data('Dataset/', '*Subject11*Session1*Coding*')
code_data = transform_ICA(code_data)

corrected_code = do_baseline(code_data, '*Subject11*Session1*Baseline*') 

med_data = get_sample_data('Dataset/', '*Subject11*Session2*Med*')
med_data = transform_ICA(med_data)

corrected_meditation = do_baseline(med_data, '*Subject11*Session2*Baseline*')

Dataset\OpenBCI-RAW-2019-06-14_15-10-10_Subject11_Session1_Coding.txt
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 413 samples (1.652 sec)

Creating RawArray with float64 data, n_channels=8, n_times=133000
    Range : 0 ... 132999 =      0.000 ...   531.996 secs
Ready.
532 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 532 events and 176 original time points ...
1 bad epochs dropped
Fitting ICA to data using 8 channels (please be patient, this may take a while)


In [114]:
spatial.distance.correlation(normalize(corrected_code[:,np.newaxis], axis=0).ravel(), normalize(corrected_meditation[:,np.newaxis], axis=0).ravel())

0.9603523800060275

In [118]:
print(compare_baseline(code_data, '*Subject11*Session1*Baseline*'))
print(compare_baseline(med_data, '*Subject11*Session2*Baseline*'))

Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 413 samples (1.652 sec)

Creating RawArray with float64 data, n_channels=8, n_times=20000
    Range : 0 ... 19999 =      0.000 ...    79.996 secs
Ready.
80 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 80 events and 176 original time points ...
1 bad epochs dropped
1988.627135064506
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal ban

In [84]:
codemed_data = get_sample_data('Dataset/', '*Subject11*Session2*Coding*')
codemed_data = transform_ICA(codemed_data)

corrected_codemed = do_baseline(codemed_data, '*Subject11*Session2*Baseline*') 

Dataset\OpenBCI-RAW-2019-06-17_19-18-48Subject11_Session2_Coding.txt
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 413 samples (1.652 sec)

Creating RawArray with float64 data, n_channels=8, n_times=133000
    Range : 0 ... 132999 =      0.000 ...   531.996 secs
Ready.
532 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 532 events and 176 original time points ...
1 bad epochs dropped
Fitting ICA to data using 8 channels (please be patient, this may take a while)
I

In [123]:
spatial.distance.euclidean(normalize(corrected_code[:,np.newaxis], axis=0).ravel(), normalize(corrected_meditation[:,np.newaxis], axis=0).ravel())

1.4723051923343229

In [119]:
compare_baseline(codemed_data, '*Subject11*Session2*Baseline*')

Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 413 samples (1.652 sec)

Creating RawArray with float64 data, n_channels=8, n_times=20000
    Range : 0 ... 19999 =      0.000 ...    79.996 secs
Ready.
80 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 80 events and 176 original time points ...
1 bad epochs dropped


190.3834848515039

In [135]:
raw_codings, baselines = get_sample_data('Dataset/', '*Session1*Coding*')

Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 413 samples (1.652 sec)

Creating RawArray with float64 data, n_channels=8, n_times=96488
    Range : 0 ... 96487 =      0.000 ...   385.948 secs
Ready.
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff 

In [136]:
mean = 0
for datas in zip(raw_codings, baselines):
    data = transform_ICA(datas[0])
    mean += compare_baseline(data, datas[1])
    
print(mean / len(raw_codings))

Fitting ICA to data using 8 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 8
Fitting ICA took 0.1s.
Transforming to ICA space (8 components)
Zeroing out 0 ICA components
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 413 samples (1.652 sec)

Creating RawArray with float64 data, n_channels=8, n_times=14519
    Range : 0 ... 14518 =      0.000 ...    58.072 secs
Ready.
58 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items acti

In [139]:
raw_codings, baselines = get_sample_data('Dataset/', '*Session2*Codingmed*')

Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 413 samples (1.652 sec)

Creating RawArray with float64 data, n_channels=8, n_times=133000
    Range : 0 ... 132999 =      0.000 ...   531.996 secs
Ready.
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutof

In [140]:
mean_med = 0
for datas in zip(raw_codings, baselines):
    data = transform_ICA(datas[0])
    mean_med += compare_baseline(data, datas[1])
    
print(mean_med / len(raw_codings))

Fitting ICA to data using 8 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 8
Fitting ICA took 0.7s.
Transforming to ICA space (8 components)
Zeroing out 0 ICA components
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 413 samples (1.652 sec)

Creating RawArray with float64 data, n_channels=8, n_times=20000
    Range : 0 ... 19999 =      0.000 ...    79.996 secs
Ready.
80 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items acti